In [ ]:
# Celda 0.1 – Imports y configuración de paths correctos

import os
import random
from datetime import date, timedelta

import numpy as np
import pandas as pd
import yaml

# -------------------------------------------------------------------
# Rutas REALES según tu proyecto:
# notebooks/
#   └── 01_generate_stde_v3.ipynb
# data/
#     ├── oms/
#     ├── ontology/
#     └── synthetic/
# -------------------------------------------------------------------

RUTA_OMS = "../data/oms/99_oms_stde_v3.yaml"

# Catálogos en ontology/
PATH_ROLES     = "../data/ontology/10_catalogo_roles_v3.yaml"
PATH_TAREAS    = "../data/ontology/12_catalogo_tareas_v1.yaml"
PATH_AREAS     = "../data/ontology/13_catalogo_areas_operacionales_v1.yaml"
PATH_RIESGOS   = "../data/ontology/01_catalogo_riesgos_v8.yaml"
PATH_CONTROLES = "../data/ontology/02_catalogo_controles_v6.yaml"

# Modelo proactivo en oms/
PATH_PROACTIVO = "../data/oms/stde_proactivo_semanal_v4_4.csv"

# Directorio de salida (synthetic/)
OUTPUT_DIR = "../data/synthetic/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Configuración de aleatoriedad
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)



In [ ]:
# Celda 0.2 – Cargar OMS v3

def cargar_yaml(path):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

oms = cargar_yaml(PATH_OMS)

# Accesos rápidos a secciones relevantes de la OMS v3
meta = oms.get("meta", {})
stde_cfg = oms.get("stde", {})
fdo_cfg = oms.get("fdo", {})
escenario_cfg = oms.get("escenario_operacion", {})
mapeos_operacionales = oms.get("mapeos_operacionales", {})
proactivo_cfg = oms.get("proactivo", {})

lunes_critico_cfg = escenario_cfg.get("lunes_critico", {})
eventos_lunes_critico = lunes_critico_cfg.get("eventos_sinteticos", [])

print("Versión OMS:", meta.get("version"))
print("Fecha inicio escenario:", escenario_cfg.get("fecha_inicio"))
print("Número de semanas:", escenario_cfg.get("numero_semanas"))
print("Eventos del lunes crítico:", len(eventos_lunes_critico))


In [ ]:
# Celda 0.3 – Cargar catálogos de ontología (roles, tareas, áreas, riesgos, controles)

def cargar_catalogo_roles(path):
    data = cargar_yaml(path)
    # Asumimos que el YAML tiene una lista principal "roles"
    roles = data.get("roles", data)  # fallback por si la raíz ya es lista
    return pd.DataFrame(roles)

def cargar_catalogo_tareas(path):
    data = cargar_yaml(path)
    tareas = data.get("tareas", data)
    return pd.DataFrame(tareas)

def cargar_catalogo_areas(path):
    data = cargar_yaml(path)
    areas = data.get("areas_operacionales", data.get("areas", data))
    return pd.DataFrame(areas)

def cargar_catalogo_riesgos(path):
    data = cargar_yaml(path)
    riesgos = data.get("riesgos", data)
    return pd.DataFrame(riesgos)

def cargar_catalogo_controles(path):
    data = cargar_yaml(path)
    controles = data.get("controles", data)
    return pd.DataFrame(controles)

df_roles = cargar_catalogo_roles(PATH_ROLES)
df_tareas = cargar_catalogo_tareas(PATH_TAREAS)
df_areas = cargar_catalogo_areas(PATH_AREAS)
df_riesgos = cargar_catalogo_riesgos(PATH_RIESGOS)
df_controles = cargar_catalogo_controles(PATH_CONTROLES)

print("Roles:", df_roles.shape, "Tareas:", df_tareas.shape, "Áreas:", df_areas.shape)
print("Riesgos:", df_riesgos.shape, "Controles:", df_controles.shape)


In [ ]:
# Celda 1.1 – Generar calendario diario del escenario STDE (12 semanas) + marca de lunes crítico

def generar_calendario(escenario_cfg):
    fecha_inicio_str = escenario_cfg["fecha_inicio"]
    numero_semanas = escenario_cfg["numero_semanas"]
    fecha_lunes_critico_str = escenario_cfg["lunes_critico"]["fecha"]

    fecha_inicio = date.fromisoformat(fecha_inicio_str)
    dias_totales = numero_semanas * 7  # 12 semanas → 84 días (sin lunes crítico)
    fecha_lunes_critico = date.fromisoformat(fecha_lunes_critico_str)

    registros = []
    for i in range(dias_totales):
        fecha = fecha_inicio + timedelta(days=i)
        semana = (i // 7) + 1  # semana 1..12
        dia_semana = fecha.weekday()  # 0 = lunes, 6 = domingo

        registros.append(
            {
                "id_dia": i + 1,
                "fecha": fecha,
                "semana": semana,
                "dia_semana": dia_semana,
                "es_lunes_critico": (fecha == fecha_lunes_critico),
            }
        )

    calendario = pd.DataFrame(registros)
    return calendario

calendario = generar_calendario(escenario_cfg)
print("Días calendario:", len(calendario))
calendario.head()


In [ ]:
# Celda 1.2 – Generar FDO diario (7 factores) con patrones suaves

fdo_factores = fdo_cfg.get("factores", [])

# Creamos un índice de factores por id para referencia si se requiere después
FDO_IDS = [f["id"] for f in fdo_factores]

def generar_fdo_diario(calendario):
    registros = []
    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # Base de variación semanal: degradación leve hacia semana 12
        # Normalizamos semana 1..12 a 0..1
        t = (semana - 1) / (calendario["semana"].max() - 1)

        # Definimos patrones simplificados coherentes con la narrativa:
        # - Producción: crece con el tiempo (más presión)
        # - Backlog: crece (mantenimiento atrasado)
        # - Congestión: crece con producción
        # - Fatiga: crece, modulada por día de la semana (más alta al final de la semana)
        # - Clima: variación aleatoria suave
        # - Dotación: algo decreciente (baja levemente)
        # - Variabilidad: aumenta moderadamente
        dow = row["dia_semana"]  # 0 lunes, 6 domingo

        fdo_produccion = np.clip(0.3 + 0.6 * t + np.random.normal(0, 0.05), 0.0, 1.0)
        fdo_backlog = np.clip(0.2 + 0.7 * t + np.random.normal(0, 0.05), 0.0, 1.0)
        fdo_congestion = np.clip(0.25 + 0.6 * t + np.random.normal(0, 0.05), 0.0, 1.0)

        # Fatiga: más alta jueves-viernes (3,4)
        fatiga_semana = 0.2 + 0.6 * t
        ajuste_dow = 0.1 if dow in (3, 4) else 0.0
        fdo_fatiga = np.clip(fatiga_semana + ajuste_dow + np.random.normal(0, 0.05), 0.0, 1.0)

        # Clima: ruido con tendencia leve
        fdo_clima = np.clip(0.4 + 0.2 * np.sin(2 * np.pi * t) + np.random.normal(0, 0.1), 0.0, 1.0)

        # Dotación: leve caída con el tiempo (renuncias, licencias, etc.)
        fdo_dotacion = np.clip(0.9 - 0.3 * t + np.random.normal(0, 0.05), 0.0, 1.0)

        # Variabilidad: aumenta hacia el final (más cambios de plan)
        fdo_variabilidad = np.clip(0.2 + 0.6 * t + np.random.normal(0, 0.05), 0.0, 1.0)

        registros.append(
            {
                "id_dia": id_dia,
                "fecha": fecha,
                "semana": semana,
                "fdo_produccion": fdo_produccion,
                "fdo_backlog": fdo_backlog,
                "fdo_congestion": fdo_congestion,
                "fdo_fatiga": fdo_fatiga,
                "fdo_clima": fdo_clima,
                "fdo_dotacion": fdo_dotacion,
                "fdo_variabilidad": fdo_variabilidad,
            }
        )

    df_fdo = pd.DataFrame(registros)
    return df_fdo

df_fdo_diario = generar_fdo_diario(calendario)
print("FDO diario generado:", df_fdo_diario.shape)
df_fdo_diario.head()


In [ ]:
# Celda 2.1 – Generar trayectorias diarias de criticidad para R01, R02, R03

RIESGOS_FOCO = ["R01", "R02", "R03"]

def generar_trayectorias_riesgo(calendario):
    registros = []
    max_semana = calendario["semana"].max()

    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # Normalización temporal 0..1 sobre las 12 semanas
        t = (semana - 1) / (max_semana - 1)

        # -------------------------------
        # R02 – Degradación acumulada fuerte
        # -------------------------------
        base_R02 = 0.35 + 0.5 * t  # crece fuertemente hacia el final
        ruido_R02 = np.random.normal(0, 0.03)
        criticidad_R02 = np.clip(base_R02 + ruido_R02, 0.0, 1.0)

        # -------------------------------
        # R01 – Controlado pero con señales débiles
        # -------------------------------
        base_R01 = 0.25 + 0.25 * t  # leve pendiente
        ruido_R01 = np.random.normal(0, 0.03)
        criticidad_R01 = np.clip(base_R01 + ruido_R01, 0.0, 1.0)

        # -------------------------------
        # R03 – Estable con variaciones leves
        # -------------------------------
        base_R03 = 0.2 + 0.15 * t  # leve aumento
        # Picos esporádicos: algunos días sube un poco más
        pico = 0.1 if random.random() < 0.08 else 0.0
        ruido_R03 = np.random.normal(0, 0.03)
        criticidad_R03 = np.clip(base_R03 + pico + ruido_R03, 0.0, 1.0)

        registros.append(
            {
                "id_dia": id_dia,
                "fecha": fecha,
                "semana": semana,
                "criticidad_R01": criticidad_R01,
                "criticidad_R02": criticidad_R02,
                "criticidad_R03": criticidad_R03,
            }
        )

    df_tray = pd.DataFrame(registros)
    return df_tray

df_trayectorias = generar_trayectorias_riesgo(calendario)
print("Trayectorias generadas:", df_trayectorias.shape)
df_trayectorias.head()


In [ ]:
# Celda 2.2 – Trayectoria ponderada global por día y agregados por semana

PESOS_RIESGO_GLOBAL = {
    "R01": 0.25,
    "R02": 0.5,
    "R03": 0.25,
}

def agregar_trayectoria_global(df_tray):
    df = df_tray.copy()

    df["criticidad_global"] = (
        df["criticidad_R01"] * PESOS_RIESGO_GLOBAL["R01"]
        + df["criticidad_R02"] * PESOS_RIESGO_GLOBAL["R02"]
        + df["criticidad_R03"] * PESOS_RIESGO_GLOBAL["R03"]
    )

    # Agregados semanales para facilitar gráficos en la demo (medias por semana)
    df_semana = (
        df.groupby("semana", as_index=False)[
            [
                "criticidad_R01",
                "criticidad_R02",
                "criticidad_R03",
                "criticidad_global",
            ]
        ]
        .mean()
        .rename(
            columns={
                "criticidad_R01": "criticidad_R01_media",
                "criticidad_R02": "criticidad_R02_media",
                "criticidad_R03": "criticidad_R03_media",
                "criticidad_global": "criticidad_global_media",
            }
        )
    )

    return df, df_semana

df_trayectorias, df_trayectorias_semana = agregar_trayectoria_global(df_trayectorias)

print("df_trayectorias:", df_trayectorias.shape)
print("df_trayectorias_semana:", df_trayectorias_semana.shape)
df_trayectorias_semana.head()


In [ ]:
# Celda 3.1 – Funciones base de probabilidad para eventos diarios STDE v3

def base_PD(criticidad):
    """
    HAZARD ~ señales débiles, más frecuentes.
    0.5 * criticidad → moderada sensibilidad.
    """
    return np.clip(0.10 + 0.50 * criticidad, 0.0, 0.9)


def base_NM(criticidad):
    """
    NMS ~ señales intermedias.
    Más fuerte en R02 (alta degradación).
    """
    return np.clip(0.02 + 0.35 * criticidad, 0.0, 0.7)


def base_IM(criticidad):
    """
    IMEN (lesión leve) ~ baja probabilidad, solo casos severos.
    """
    return np.clip(0.005 + 0.08 * criticidad, 0.0, 0.25)


def modular_por_fdo(prob, fdo_row):
    """
    Ajusta la probabilidad con los FDO diarios.
    Reglas STDE v3:
      - producción, congestión y fatiga aumentan prob. de eventos
      - dotación baja → aumenta riesgo
      - clima extremo → altera probabilidad
    """
    factor = (
        1.0
        + 0.30 * fdo_row["fdo_produccion"]
        + 0.20 * fdo_row["fdo_congestion"]
        + 0.20 * fdo_row["fdo_fatiga"]
        + 0.10 * (1.0 - fdo_row["fdo]()_*


In [ ]:
# Celda 3.1b – Versión corregida de modular_por_fdo (redefine solo esta función)

def modular_por_fdo(prob, fdo_row):
    """
    Ajusta la probabilidad con los FDO diarios.
    Reglas STDE v3:
      - producción, congestión y fatiga aumentan prob. de eventos
      - dotación baja → aumenta riesgo
      - clima extremo → altera probabilidad
    """
    factor = (
        1.0
        + 0.30 * fdo_row["fdo_produccion"]
        + 0.20 * fdo_row["fdo_congestion"]
        + 0.20 * fdo_row["fdo_fatiga"]
        + 0.10 * (1.0 - fdo_row["fdo_dotacion"])   # menos dotación → más riesgo
        + 0.10 * abs(fdo_row["fdo_clima"] - 0.5)   # clima muy bueno o muy malo altera el riesgo
    )

    prob_mod = np.clip(prob * factor, 0.0, 0.99)
    return prob_mod


In [ ]:
# Celda 3.2 – Generar eventos diarios (HAZARD / NMS / IMEN) coherentes con OMS v3

# Extraer mapeo riesgo_por_area
riesgo_por_area = mapeos_operacionales.get("riesgo_por_area", {})

# Preprocesar roles y tareas por área
roles_por_area = df_roles.groupby("area_operacional_id")["id_rol"].apply(list).to_dict()
tareas_por_area = df_tareas.groupby("area_operacional_id")["id_tarea"].apply(list).to_dict()


def elegir_riesgo_area(area_id):
    """
    Selecciona un riesgo según los pesos definidos en la OMS.
    """
    lista = riesgo_por_area.get(area_id, [])
    if not lista:
        return None

    riesgos = [x["riesgo_id"] for x in lista]
    pesos = [x["peso_relativo"] for x in lista]
    return random.choices(riesgos, weights=pesos, k=1)[0]


def asignar_tipo_incidente(prob_pd, prob_nm, prob_im):
    """
    Según probabilidades ajustadas, asigna el tipo de incidente narrativo:
    HAZARD, NMS, IMEN.
    """
    r = random.random()
    if r < prob_im:
        return "IMEN"
    elif r < (prob_nm + prob_im):
        return "NMS"
    elif r < (prob_pd + prob_nm + prob_im):
        return "HAZARD"
    else:
        return None


def mapear_tipo_formal(tipo_narrativo):
    """
    Mapea narrativa → formal según OMS v3:
        HAZARD → INMI
        NMS    → INMI
        IMEN   → ILEV
    """
    if tipo_narrativo in ["HAZARD", "NMS"]:
        return "INMI"
    if tipo_narrativo == "IMEN":
        return "ILEV"
    return None


def generar_eventos_v3(calendario, df_tray, df_fdo, oms):

    registros = []

    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # Filtrar FDO y trayectorias del día
        fdo_row = df_fdo[df_fdo["id_dia"] == id_dia].iloc[0]
        tray_row = df_tray[df_tray["id_dia"] == id_dia].iloc[0]

        for _, area_row in df_areas.iterrows():
            area_id = area_row["id_area"]
            
            # 1. Seleccionar el riesgo dominante del área
            riesgo_id = elegir_riesgo_area(area_id)
            if riesgo_id not in ["R01", "R02", "R03"]:
                continue  # solo modelamos 3 riesgos

            criticidad = tray_row[f"criticidad_{riesgo_id}"]

            # 2. Probabilidades base
            prob_pd = base_PD(criticidad)
            prob_nm = base_NM(criticidad)
            prob_im = base_IM(criticidad)

            # 3. Modulación FDO
            prob_pd = modular_por_fdo(prob_pd, fdo_row)
            prob_nm = modular_por_fdo(prob_nm, fdo_row)
            prob_im = modular_por_fdo(prob_im, fdo_row)

            # 4. Determinar si ocurre evento
            tipo_narrativo = asignar_tipo_incidente(prob_pd, prob_nm, prob_im)
            if tipo_narrativo is None:
                continue

            tipo_formal = mapear_tipo_formal(tipo_narrativo)

            # 5. Rol y tarea coherentes
            rol_ids = roles_por_area.get(area_id, [])
            tarea_ids = tareas_por_area.get(area_id, [])
            rol_id = random.choice(rol_ids) if rol_ids else None
            tarea_id = random.choice(tarea_ids) if tarea_ids else None

            registros.append({
                "id_evento": f"EVT_{id_dia}_{area_id}_{riesgo_id}",
                "fecha": fecha,
                "semana": semana,
                "id_area": area_id,
                "rol_id": rol_id,
                "tarea_id": tarea_id,
                "riesgo_id": riesgo_id,
                "tipo_narrativo": tipo_narrativo,  # HAZARD / NMS / IMEN
                "tipo_formal": tipo_formal,        # INMI / ILEV
                "criticidad": criticidad,
                "prob_pd": prob_pd,
                "prob_nm": prob_nm,
                "prob_im": prob_im,
                "es_lunes_critico": False,
            })

    df_eventos = pd.DataFrame(registros)
    return df_eventos


df_eventos = generar_eventos_v3(calendario, df_trayectorias, df_fdo_diario, oms)
print("Eventos generados:", df_eventos.shape)
df_eventos.head()


In [ ]:
# Celda 3.3 – Inyección del lunes crítico (evento compuesto real)

def inyectar_lunes_critico(df_eventos, calendario):
    """
    Inserta el lunes crítico como un EVENTO COMPUESTO:
    - 1 registro maestro (LC_EVT_00)
    - 2 sub-eventos NMS asociados (R01 y R02)
    Usando EXACTAMENTE la narrativa del archivo 'Evento lunes critico.txt'.
    """

    # Fecha y semana del LC
    fecha_lc = calendario[calendario["es_lunes_critico"] == True]["fecha"].iloc[0]
    semana_lc = calendario[calendario["es_lunes_critico"] == True]["semana"].iloc[0]

    registros = []

    # =============================================================
    # EVENTO MAESTRO — Evento compuesto
    # =============================================================
    registros.append({
        "id_evento": "LC_EVT_00",
        "id_evento_compuesto": None,
        "fecha": fecha_lc,
        "semana": semana_lc,
        "id_area": "MRA",
        "rol_id": None,
        "tarea_id": None,

        "riesgo_id": None,
        "tipo_narrativo": "NMS_COMPUESTO",
        "tipo_formal": "INMI",

        "criticidad": 0.92,  # outlier realista
        "prob_pd": None,
        "prob_nm": None,
        "prob_im": None,
        "es_lunes_critico": True,

        # NARRATIVA EXACTA
        "descripcion_evento": (
            "Durante la inspección rutinaria en un andamio móvil, un trabajador perdió parcialmente "
            "el equilibrio por una irregularidad en la plataforma y, en la maniobra, también se le "
            "cayó una llave desde altura. El arnés detuvo la caída y la herramienta no impactó "
            "a ninguna persona."
        ),

        "evento_principal_ocurrido": (
            "Pérdida de equilibrio en andamio que activa simultáneamente caída detenida (R01) "
            "y caída de objeto (R02)."
        ),

        "control_critico_relacionado": None,
        "potencial_consecuencia_severidad": None,
    })

    # =============================================================
    # SUB-EVENTO R01 — Caída detenida por arnés
    # =============================================================
    registros.append({
        "id_evento": "LC_EVT_01",
        "id_evento_compuesto": "LC_EVT_00",
        "fecha": fecha_lc,
        "semana": semana_lc,
        "id_area": "MRA",
        "rol_id": None,
        "tarea_id": None,

        "riesgo_id": "R01",
        "tipo_narrativo": "NMS",
        "tipo_formal": "INMI",

        "criticidad": 0.90,
        "prob_pd": None,
        "prob_nm": None,
        "prob_im": None,
        "es_lunes_critico": True,

        "descripcion_evento": (
            "Trabajador pierde equilibrio en andamio. El arnés de seguridad activa el sistema "
            "de detención evitando consecuencias mayores."
        ),

        "evento_principal_ocurrido": "Caída detenida por el arnés.",
        "control_critico_relacionado": "R01CC01",
        "potencial_consecuencia_severidad": "SEV4",
    })

    # =============================================================
    # SUB-EVENTO R02 — Caída de llave sin impacto
    # =============================================================
    registros.append({
        "id_evento": "LC_EVT_02",
        "id_evento_compuesto": "LC_EVT_00",
        "fecha": fecha_lc,
        "semana": semana_lc,
        "id_area": "MRA",
        "rol_id": None,
        "tarea_id": None,

        "riesgo_id": "R02",
        "tipo_narrativo": "NMS",
        "tipo_formal": "INMI",

        "criticidad": 0.85,
        "prob_pd": None,
        "prob_nm": None,
        "prob_im": None,
        "es_lunes_critico": True,

        "descripcion_evento": (
            "Durante la pérdida de equilibrio, una llave cayó desde ~2.5 metros sin impactar "
            "a personas, registrándose como Near Miss."
        ),

        "evento_principal_ocurrido": "Caída de objeto desde altura.",
        "control_critico_relacionado": "R02CC03",
        "potencial_consecuencia_severidad": "SEV3",
    })

    # =============================================================
    # Integrar con df_eventos existente
    # =============================================================
    df_lc = pd.DataFrame(registros)
    df_final = pd.concat([df_eventos, df_lc], ignore_index=True)

    return df_final


# Aplicar inyección
df_eventos = inyectar_lunes_critico(df_eventos, calendario)

print("Eventos totales con lunes crítico:", df_eventos.shape)
df_eventos[df_eventos["es_lunes_critico"] == True]


In [ ]:
# Celda 4.1 – Generación de OPG (ruido operacional)

# Tasas por área (según OMS v3 - regla de ruido operacional)
tasa_opg_area = {
    "MRA": (2, 4),
    "PLC": (1, 3),
    "TRN": (1, 2),
    "TME": (1, 2),
}

def generar_opg(calendario, df_fdo, df_tray, df_roles, df_tareas):

    registros = []

    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        fdo_row = df_fdo[df_fdo["id_dia"] == id_dia].iloc[0]
        tray_row = df_tray[df_tray["id_dia"] == id_dia].iloc[0]

        for _, area in df_areas.iterrows():
            area_id = area["id_area"]

            # 1. Cuántas OPG generar hoy
            low, high = tasa_opg_area.get(area_id, (1, 2))
            n_opg = random.randint(low, high)

            # 2. Riesgo dominante por área → afecta positividad/negatividad
            try:
                riesgo_dom = elegir_riesgo_area(area_id)
            except:
                riesgo_dom = None

            criticidad = tray_row[f"criticidad_{riesgo_dom}"] if riesgo_dom in ["R01","R02","R03"] else 0.2

            for _ in range(n_opg):

                rol_id = random.choice(roles_por_area.get(area_id, [None]))
                tarea_id = random.choice(tareas_por_area.get(area_id, [None]))

                # Probabilidad de OPG negativa (OPG–)
                p_neg = (
                    0.10
                    + 0.30 * fdo_row["fdo_fatiga"]
                    + 0.20 * fdo_row["fdo_congestion"]
                    + 0.30 * criticidad
                )

                estado = "OPG-" if random.random() < p_neg else "OPG+"

                registros.append({
                    "id_observacion": f"OPG_{id_dia}_{area_id}_{random.randint(1000,9999)}",
                    "fecha": fecha,
                    "semana": semana,
                    "id_area": area_id,
                    "rol_observador_id": rol_id,
                    "rol_observado_id": None,
                    "tarea_id": tarea_id,
                    "tipo_observacion": "OPG",
                    "estado": estado,
                    "riesgo_id": riesgo_dom,
                    "is_control_critico": False,
                    "control_critico_id": None,
                })

    return pd.DataFrame(registros)


df_opg = generar_opg(calendario, df_fdo_diario, df_trayectorias, df_roles, df_tareas)
print("OPG generadas:", df_opg.shape)
df_opg.head()


In [ ]:
# Celda 4.2 – Generación de OCC reactivas

# Diccionario de controles críticos por riesgo
controles_por_riesgo = df_controles.groupby("riesgo_asociado")["id"].apply(list).to_dict()

def generar_occ(df_eventos, calendario, df_fdo):

    registros = []

    for _, ev in df_eventos.iterrows():

        if ev["tipo_narrativo"] not in ["HAZARD", "NMS", "IMEN"]:
            continue

        fecha = ev["fecha"]
        semana = ev["semana"]
        riesgo_id = ev["riesgo_id"]
        id_dia = calendario[calendario["fecha"] == fecha]["id_dia"].iloc[0]

        fdo_row = df_fdo[df_fdo["id_dia"] == id_dia].iloc[0]

        # 1 o 2 OCC por evento
        n_occ = random.randint(1, 2)

        # picks del área → roles y tareas
        area_id = ev["id_area"]
        rol_id = random.choice(roles_por_area.get(area_id, [None]))
        tarea_id = random.choice(tareas_por_area.get(area_id, [None]))

        # control crítico asociado
        posibles_controles = controles_por_riesgo.get(riesgo_id, [])
        control_usado = random.choice(posibles_controles) if posibles_controles else None

        for _ in range(n_occ):

            # Probabilidad de OCC negativo
            p_neg = (
                0.15
                + 0.25 * fdo_row["fdo_fatiga"]
                + 0.20 * fdo_row["fdo_congestion"]
                + 0.30 * ev["criticidad"]
            )

            # EXCEPCIÓN: Lunes crítico R01 → OCC+ explícito
            if ev.get("es_lunes_critico") and riesgo_id == "R01":
                estado = "OCC+"
            else:
                estado = "OCC-" if random.random() < p_neg else "OCC+"

            registros.append({
                "id_observacion": f"OCC_{ev['id_evento']}_{random.randint(1000,9999)}",
                "fecha": fecha,
                "semana": semana,
                "id_area": area_id,
                "rol_observador_id": rol_id,
                "rol_observado_id": None,
                "tarea_id": tarea_id,
                "tipo_observacion": "OCC",
                "estado": estado,
                "riesgo_id": riesgo_id,
                "is_control_critico": True,
                "control_critico_id": control_usado,
                "id_evento_origen": ev["id_evento"],
            })

    return pd.DataFrame(registros)


df_occ = generar_occ(df_eventos, calendario, df_fdo_diario)
print("OCC generadas:", df_occ.shape)
df_occ.head()


In [ ]:
# Celda 4.3 – Consolidar observaciones OPG + OCC

df_observaciones = pd.concat([df_opg, df_occ], ignore_index=True)
print("Total observaciones:", df_observaciones.shape)
df_observaciones.head()


In [ ]:
# Celda 5.1 – Auditorías AUF reactivas

def generar_auditorias_reactivas(df_eventos, df_observaciones, df_fdo, calendario):

    registros = []

    # ------------------------------------------------------------------
    # 1. AUF por Near Miss severo (SEV3 o SEV4)
    # ------------------------------------------------------------------
    df_nms = df_eventos[
        (df_eventos["tipo_narrativo"] == "NMS") &
        (df_eventos["potencial_consecuencia_severidad"].isin(["SEV3", "SEV4"]))
    ]

    for _, ev in df_nms.iterrows():
        fecha_ev = ev["fecha"]

        # AUF programada máximo 48 horas después
        fecha_auf = fecha_ev + timedelta(days=random.choice([1, 2]))
        semana_auf = calendario[calendario["fecha"] == fecha_auf]["semana"]
        semana_auf = int(semana_auf.iloc[0]) if not semana_auf.empty else ev["semana"]

        registros.append({
            "id_auditoria": f"AUF_NMS_{ev['id_evento']}",
            "fecha": fecha_auf,
            "semana": semana_auf,
            "id_area": ev["id_area"],
            "tipo_auditoria": "AUF",
            "origen": "post_nm",
            "riesgo_focal": ev["riesgo_id"],
            "id_evento_origen": ev["id_evento"],
        })

    # ------------------------------------------------------------------
    # 2. AUF por falla de control crítico (OCC−)
    # ------------------------------------------------------------------
    df_occ_neg = df_observaciones[
        (df_observaciones["tipo_observacion"] == "OCC") &
        (df_observaciones["estado"] == "OCC-")
    ]

    for _, obs in df_occ_neg.iterrows():
        fecha_obs = obs["fecha"]
        semana_obs = obs["semana"]

        registros.append({
            "id_auditoria": f"AUF_OCCNEG_{obs['id_observacion']}",
            "fecha": fecha_obs,
            "semana": semana_obs,
            "id_area": obs["id_area"],
            "tipo_auditoria": "AUF",
            "origen": "post_incidente",
            "riesgo_focal": obs["riesgo_id"],
            "id_evento_origen": obs.get("id_evento_origen"),
        })

    # ------------------------------------------------------------------
    # 3. AUF por señales PD críticas dos días seguidos
    #    (criticidad diaria > 0.75 por dos días consecutivos)
    # ------------------------------------------------------------------
    df_fdo_sorted = df_fdo.sort_values("id_dia")

    for riesgo in ["R01", "R02", "R03"]:
        criticidades = df_eventos.groupby("fecha")["criticidad"].mean()

        fechas = list(criticidades.index)
        for i in range(len(fechas) - 1):
            if criticidades.iloc[i] > 0.75 and criticidades.iloc[i + 1] > 0.75:

                fecha_auf = fechas[i + 1]
                semana_auf = calendario[calendario["fecha"] == fecha_auf]["semana"].iloc[0]

                registros.append({
                    "id_auditoria": f"AUF_PDCRIT_{riesgo}_{i}",
                    "fecha": fecha_auf,
                    "semana": semana_auf,
                    "id_area": "MRA",
                    "tipo_auditoria": "AUF",
                    "origen": "post_pd_critico",
                    "riesgo_focal": riesgo,
                    "id_evento_origen": None,
                })

    # ------------------------------------------------------------------
    # 4. AUF por el lunes crítico (forzadas por narrativa)
    # ------------------------------------------------------------------
    df_lc = df_eventos[df_eventos["es_lunes_critico"] == True]

    for _, ev in df_lc.iterrows():
        registros.append({
            "id_auditoria": f"AUF_LC_{ev['id_evento']}",
            "fecha": ev["fecha"],
            "semana": ev["semana"],
            "id_area": ev["id_area"],
            "tipo_auditoria": "AUF",
            "origen": "post_nm_compuesto",
            "riesgo_focal": ev["riesgo_id"],
            "id_evento_origen": ev["id_evento"],
        })

    return pd.DataFrame(registros)


df_auf_reactivas = generar_auditorias_reactivas(df_eventos, df_observaciones, df_fdo_diario, calendario)
print("AUF reactivas generadas:", df_auf_reactivas.shape)
df_auf_reactivas.head()


In [ ]:
# Celda 5.2 – Auditorías AUF planificadas (cierre de mes)

def generar_auditorias_planificadas(calendario, df_areas):

    registros = []
    meses = calendario["fecha"].dt.month.unique()

    for mes in meses:
        ultimo_dia_mes = calendario[calendario["fecha"].dt.month == mes]["fecha"].max()
        semana_mes = calendario[calendario["fecha"] == ultimo_dia_mes]["semana"].iloc[0]

        for _, area in df_areas.iterrows():

            registros.append({
                "id_auditoria": f"AUF_PLAN_{area['id_area']}_{mes}",
                "fecha": ultimo_dia_mes,
                "semana": semana_mes,
                "id_area": area["id_area"],
                "tipo_auditoria": "AUF",
                "origen": "cierre_mes",
                "riesgo_focal": None,
                "id_evento_origen": None,
            })

    return pd.DataFrame(registros)


df_auf_planificadas = generar_auditorias_planificadas(calendario, df_areas)
print("AUF planificadas:", df_auf_planificadas.shape)
df_auf_planificadas.head()


In [ ]:
# Celda 5.3 – Consolidación AUF reactivas + planificadas

df_auditorias = pd.concat(
    [df_auf_reactivas, df_auf_planificadas],
    ignore_index=True
)

print("Total auditorías generadas:", df_auditorias.shape)
df_auditorias.head()


In [ ]:
# Celda 6.1 – Cargar modelo proactivo v4.4

df_proactivo = pd.read_csv(PATH_PROACTIVO)

# Validación rápida
print("Proactivo v4.4 cargado:", df_proactivo.shape)
df_proactivo.head(12)


In [ ]:
# Celda 6.2 – Calcular thresholds por semana y riesgo

def calcular_thresholds(df_proactivo):

    registros = []

    for semana in sorted(df_proactivo["semana_id"].unique()):
        df_sem = df_proactivo[df_proactivo["semana_id"] == semana]

        scores = df_sem["score_proactivo"].values

        # Thresholds basados en percentiles (modelo real)
        thr_alerta  = np.round(np.percentile(scores, 60), 2)
        thr_critico = np.round(np.percentile(scores, 80), 2)

        for _, row in df_sem.iterrows():
            registros.append({
                "semana_id": semana,
                "riesgo_id": row["riesgo_id"],
                "score_proactivo": row["score_proactivo"],
                "rank_proactivo": row["rank_proactivo"],
                "threshold_alerta": thr_alerta,
                "threshold_critico": thr_critico
            })

    return pd.DataFrame(registros)


df_proactivo_thresholds = calcular_thresholds(df_proactivo)
print("Thresholds generados:", df_proactivo_thresholds.shape)
df_proactivo_thresholds.head(15)


In [ ]:
# Celda 6.3 – Comparar trayectorias reales vs modelo proactivo

# 1. Agregar criticidad semanal real (STDE)
df_tray_sem = (
    df_trayectorias
    .groupby("semana")[["criticidad_R01","criticidad_R02","criticidad_R03"]]
    .mean()
    .reset_index()
    .rename(columns={"semana":"semana_id"})
)

# 2. Agregar thresholds semanales
df_compare = df_proactivo_thresholds.merge(df_tray_sem, on="semana_id", how="left")

# 3. Filtrar solo los riesgos modelados STDE v3
df_compare_stde = df_compare[df_compare["riesgo_id"].isin(["R01","R02","R03"])]

# Mostrar evidencia clara de desalineación
df_compare_stde_sorted = df_compare_stde.sort_values(["semana_id","rank_proactivo"])

df_compare_stde_sorted.head(15)


In [ ]:
# Celda 6.4 – Señales individuales por riesgo y semana (base para índice K9)

RIESGOS_FOCO = ["R01", "R02", "R03"]

def construir_senales_riesgo_semana(
    df_trayectorias,
    df_eventos,
    df_observaciones,
    df_auditorias,
    df_fdo
):
    # 1) Criticidad semanal por riesgo (desde df_trayectorias)
    df_tray_sem = (
        df_trayectorias
        .groupby("semana")[["criticidad_R01", "criticidad_R02", "criticidad_R03"]]
        .mean()
        .reset_index()
        .rename(columns={"semana": "semana_id"})
    )

    # 2) Eventos por tipo narrativo
    df_ev_agg = (
        df_eventos
        .dropna(subset=["riesgo_id"])
        .groupby(["semana", "riesgo_id", "tipo_narrativo"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )
    # aseguremos columnas
    for col in ["HAZARD", "NMS", "IMEN"]:
        if col not in df_ev_agg.columns:
            df_ev_agg[col] = 0
    df_ev_agg = df_ev_agg.rename(columns={"semana": "semana_id"})

    # 3) Observaciones OPG / OCC por estado
    df_obs_agg = (
        df_observaciones
        .dropna(subset=["riesgo_id"])
        .groupby(["semana", "riesgo_id", "tipo_observacion", "estado"])
        .size()
        .reset_index(name="n")
    )

    def contar_obs(tipo, estado):
        df_tmp = df_obs_agg[
            (df_obs_agg["tipo_observacion"] == tipo)
            & (df_obs_agg["estado"] == estado)
        ]
        df_tmp = df_tmp.groupby(["semana", "riesgo_id"])["n"].sum().reset_index()
        df_tmp = df_tmp.rename(columns={"n": f"n_{tipo}_{estado}"})
        return df_tmp

    df_opg_pos = contar_obs("OPG", "OPG+")
    df_opg_neg = contar_obs("OPG", "OPG-")
    df_occ_pos = contar_obs("OCC", "OCC+")
    df_occ_neg = contar_obs("OCC", "OCC-")

    # Merge de observaciones
    df_obs_merge = df_opg_pos.merge(df_opg_neg, on=["semana", "riesgo_id"], how="outer")
    df_obs_merge = df_obs_merge.merge(df_occ_pos, on=["semana", "riesgo_id"], how="outer")
    df_obs_merge = df_obs_merge.merge(df_occ_neg, on=["semana", "riesgo_id"], how="outer")
    df_obs_merge = df_obs_merge.fillna(0)
    df_obs_merge = df_obs_merge.rename(columns={"semana": "semana_id"})

    # 4) Auditorías AUF por riesgo/semana
    df_aud_agg = (
        df_auditorias
        .dropna(subset=["riesgo_focal"])
        .groupby(["semana", "riesgo_focal"])
        .size()
        .reset_index(name="n_auf")
        .rename(columns={"semana": "semana_id", "riesgo_focal": "riesgo_id"})
    )

    # 5) FDO contextual semanal (mismo valor para todos los riesgos, pero útil para gráficas)
    df_fdo_sem = (
        df_fdo
        .groupby("semana")[[
            "fdo_produccion",
            "fdo_backlog",
            "fdo_congestion",
            "fdo_fatiga",
            "fdo_clima",
            "fdo_dotacion",
            "fdo_variabilidad",
        ]]
        .mean()
        .reset_index()
        .rename(columns={"semana": "semana_id"})
    )
    df_fdo_sem["fdo_contexto"] = (
        0.25 * df_fdo_sem["fdo_produccion"]
        + 0.20 * df_fdo_sem["fdo_backlog"]
        + 0.20 * df_fdo_sem["fdo_congestion"]
        + 0.20 * df_fdo_sem["fdo_fatiga"]
        + 0.15 * (1.0 - df_fdo_sem["fdo_dotacion"])
    )

    # 6) Construir tabla final por semana x riesgo
    registros = []
    for semana in sorted(df_tray_sem["semana_id"].unique()):
        for riesgo in RIESGOS_FOCO:
            row_tray = df_tray_sem[df_tray_sem["semana_id"] == semana].iloc[0]
            crit_col = f"criticidad_{riesgo}_media"
            if crit_col not in row_tray.index:
                # renombramos una sola vez: criticidad_R01 -> criticidad_R01_media, etc.
                pass

            criticidad = row_tray[f"criticidad_{riesgo}"]

            # Eventos
            row_ev = df_ev_agg[
                (df_ev_agg["semana_id"] == semana) & (df_ev_agg["riesgo_id"] == riesgo)
            ]
            if not row_ev.empty:
                n_hazard = int(row_ev["HAZARD"].iloc[0])
                n_nms = int(row_ev["NMS"].iloc[0])
                n_imen = int(row_ev["IMEN"].iloc[0])
            else:
                n_hazard = n_nms = n_imen = 0

            # Observaciones
            row_obs = df_obs_merge[
                (df_obs_merge["semana_id"] == semana) & (df_obs_merge["riesgo_id"] == riesgo)
            ]
            if not row_obs.empty:
                n_opg_pos = int(row_obs["n_OPG_OPG+"].iloc[0])
                n_opg_neg = int(row_obs["n_OPG_OPG-"].iloc[0])
                n_occ_pos = int(row_obs["n_OCC_OCC+"].iloc[0])
                n_occ_neg = int(row_obs["n_OCC_OCC-"].iloc[0])
            else:
                n_opg_pos = n_opg_neg = n_occ_pos = n_occ_neg = 0

            # Auditorías
            row_aud = df_aud_agg[
                (df_aud_agg["semana_id"] == semana) & (df_aud_agg["riesgo_id"] == riesgo)
            ]
            n_auf = int(row_aud["n_auf"].iloc[0]) if not row_aud.empty else 0

            # FDO contexto semanal
            row_fdo = df_fdo_sem[df_fdo_sem["semana_id"] == semana].iloc[0]
            fdo_contexto = float(row_fdo["fdo_contexto"])

            registros.append({
                "semana_id": semana,
                "riesgo_id": riesgo,
                "criticidad_stde": float(criticidad),
                "n_hazard": n_hazard,
                "n_nms": n_nms,
                "n_imen": n_imen,
                "n_opg_pos": n_opg_pos,
                "n_opg_neg": n_opg_neg,
                "n_occ_pos": n_occ_pos,
                "n_occ_neg": n_occ_neg,
                "n_auf": n_auf,
                "fdo_contexto": fdo_contexto,
            })

    df_senales = pd.DataFrame(registros)
    return df_senales


df_senales_k9 = construir_senales_riesgo_semana(
    df_trayectorias,
    df_eventos,
    df_observaciones,
    df_auditorias,
    df_fdo_diario,
)

print("Señales K9 por riesgo/semana:", df_senales_k9.shape)
df_senales_k9.head()


In [ ]:
# Celda 6.5 – Índice K9 (zona amarilla/roja + tendencia por riesgo y semana)

def normalizar_columna(df, col):
    max_val = df[col].max()
    if max_val is None or max_val == 0:
        return np.zeros(len(df))
    return df[col] / max_val


def construir_indice_k9(df_senales):
    df = df_senales.copy()

    # Normalización 0–1 de features
    df["criticidad_norm"] = normalizar_columna(df, "criticidad_stde")
    df["n_nms_norm"]      = normalizar_columna(df, "n_nms")
    df["n_imen_norm"]     = normalizar_columna(df, "n_imen")
    df["n_occ_neg_norm"]  = normalizar_columna(df, "n_occ_neg")
    df["n_opg_neg_norm"]  = normalizar_columna(df, "n_opg_neg")
    df["n_auf_norm"]      = normalizar_columna(df, "n_auf")

    # Índice K9: combinación ponderada (señales de degradación)
    df["indice_k9"] = (
        0.35 * df["criticidad_norm"]
        + 0.25 * df["n_nms_norm"]
        + 0.15 * df["n_imen_norm"]
        + 0.15 * df["n_occ_neg_norm"]
        + 0.05 * df["n_opg_neg_norm"]
        + 0.05 * df["n_auf_norm"]
    )

    # Zona K9 (sin zona “verde”; usamos NORMAL como base)
    thr_amarilla = 0.55
    thr_roja     = 0.75

    def clasificar_zona(val):
        if val >= thr_roja:
            return "roja"
        elif val >= thr_amarilla:
            return "amarilla"
        else:
            return "normal"

    df["zona_k9"] = df["indice_k9"].apply(clasificar_zona)

    # Tendencia (comparación semana a semana por riesgo)
    df = df.sort_values(["riesgo_id", "semana_id"]).reset_index(drop=True)
    df["delta_indice"] = 0.0
    df["tendencia_k9"] = "estable"

    for riesgo in df["riesgo_id"].unique():
        mask = df["riesgo_id"] == riesgo
        idxs = df[mask].index.tolist()
        for i in range(1, len(idxs)):
            idx_prev = idxs[i - 1]
            idx_curr = idxs[i]
            delta = df.loc[idx_curr, "indice_k9"] - df.loc[idx_prev, "indice_k9"]
            df.loc[idx_curr, "delta_indice"] = delta

            if delta >= 0.10:
                df.loc[idx_curr, "tendencia_k9"] = "acelerada"
            elif 0.03 <= delta < 0.10:
                df.loc[idx_curr, "tendencia_k9"] = "ascendente_lenta"
            elif -0.03 < delta < 0.03:
                df.loc[idx_curr, "tendencia_k9"] = "estable"
            else:
                df.loc[idx_curr, "tendencia_k9"] = "descendente"

    return df


df_k9 = construir_indice_k9(df_senales_k9)
print("Índice K9 por riesgo/semana:", df_k9.shape)
df_k9.head(12)


In [ ]:
# Celda 6.6 – Comparar modelo proactivo vs STDE vs K9 por riesgo y semana

# Partimos de df_proactivo_thresholds (ya calculado en 6.2)
# y df_k9 con índice y zonas.

# Aseguramos tipos compatibles
df_k9_compare = df_k9.rename(columns={"semana_id": "semana_id", "riesgo_id": "riesgo_id"})

df_compare_full = df_proactivo_thresholds.merge(
    df_k9_compare,
    on=["semana_id", "riesgo_id"],
    how="left"
)

# Nos quedamos con los riesgos modelados en STDE v3
df_compare_full = df_compare_full[df_compare_full["riesgo_id"].isin(RIESGOS_FOCO)]

# Orden lógico para análisis
df_compare_full = df_compare_full.sort_values(["semana_id", "riesgo_id"])

print("Comparación Proactivo vs K9:", df_compare_full.shape)
df_compare_full.head(20)


In [ ]:
# Celda 7.1 – Identificación del outlier del lunes crítico

def detectar_outlier_lunes_critico(df_k9, df_eventos):
    """
    Detecta señales previas al lunes crítico y compara con el evento real.
    Devuelve un diccionario con la interpretación K9.
    """

    # 1) Detectar la fecha del lunes crítico
    df_lc = df_eventos[df_eventos["es_lunes_critico"] == True]
    fecha_lc = df_lc["fecha"].iloc[0]
    semana_lc = df_lc["semana"].iloc[0]

    # 2) Filtrar trayectoria K9 previa para R01
    df_r01 = df_k9[df_k9["riesgo_id"] == "R01"]
    df_r01_prev = df_r01[df_r01["semana_id"] < semana_lc]

    # 3) Señales previas
    zona_prev = df_r01_prev["zona_k9"].tolist()
    tendencia_prev = df_r01_prev["tendencia_k9"].tolist()
    indice_prev = df_r01_prev["indice_k9"].tolist()

    outlier_info = {
        "fecha_lc": fecha_lc,
        "semana_lc": int(semana_lc),
        "zona_prev": zona_prev,
        "tendencia_prev": tendencia_prev,
        "indice_prev": indice_prev,
        "n_eventos_lc": df_lc.shape[0],
        "riesgos_lc": df_lc["riesgo_id"].unique().tolist(),
    }

    return outlier_info


outlier_info = detectar_outlier_lunes_critico(df_k9, df_eventos)
outlier_info


In [ ]:
# Celda 7.2 – Ajuste conceptual del índice K9 post lunes crítico

def recalibrar_k9(outlier_info, df_k9):
    """
    Define nuevos thresholds y pesos internos para el índice K9.
    No altera datos históricos: solo define parámetros futuros.
    """

    # Thresholds originales
    T_yellow = 0.55
    T_red = 0.75

    # Señales observadas previamente
    prev_zonas = outlier_info["zona_prev"]
    prev_tendencias = outlier_info["tendencia_prev"]

    # ¿Había señales previas débiles?
    senales_previas = any(z == "amarilla" for z in prev_zonas) or \
                      any(t in ["ascendente_lenta", "acelerada"] for t in prev_tendencias)

    # Ajuste de thresholds
    if senales_previas:
        # K9 se vuelve más estricto
        T_yellow_new = max(0.45, T_yellow - 0.05)
        T_red_new = max(0.65, T_red - 0.05)
    else:
        T_yellow_new = T_yellow
        T_red_new = T_red

    # Ajuste de pesos internos
    pesos_k9_new = {
        "w_criticidad": 0.30,
        "w_nms": 0.30,        # más peso a near miss
        "w_imen": 0.15,
        "w_occ_neg": 0.20,    # más sensibilidad a OCC-
        "w_opg_neg": 0.03,
        "w_auf": 0.02
    }

    return {
        "T_yellow_new": T_yellow_new,
        "T_red_new": T_red_new,
        "pesos_k9_new": pesos_k9_new
    }


ajuste_k9 = recalibrar_k9(outlier_info, df_k9)
ajuste_k9


In [ ]:
# Celda 7.2 – Ajuste conceptual del índice K9 post lunes crítico

def recalibrar_k9(outlier_info, df_k9):
    """
    Define nuevos thresholds y pesos internos para el índice K9.
    No altera datos históricos: solo define parámetros futuros.
    """

    # Thresholds originales
    T_yellow = 0.55
    T_red = 0.75

    # Señales observadas previamente
    prev_zonas = outlier_info["zona_prev"]
    prev_tendencias = outlier_info["tendencia_prev"]

    # ¿Había señales previas débiles?
    senales_previas = any(z == "amarilla" for z in prev_zonas) or \
                      any(t in ["ascendente_lenta", "acelerada"] for t in prev_tendencias)

    # Ajuste de thresholds
    if senales_previas:
        # K9 se vuelve más estricto
        T_yellow_new = max(0.45, T_yellow - 0.05)
        T_red_new = max(0.65, T_red - 0.05)
    else:
        T_yellow_new = T_yellow
        T_red_new = T_red

    # Ajuste de pesos internos
    pesos_k9_new = {
        "w_criticidad": 0.30,
        "w_nms": 0.30,        # más peso a near miss
        "w_imen": 0.15,
        "w_occ_neg": 0.20,    # más sensibilidad a OCC-
        "w_opg_neg": 0.03,
        "w_auf": 0.02
    }

    return {
        "T_yellow_new": T_yellow_new,
        "T_red_new": T_red_new,
        "pesos_k9_new": pesos_k9_new
    }


ajuste_k9 = recalibrar_k9(outlier_info, df_k9)
ajuste_k9


In [ ]:
# Celda 7.3 – Reporte narrativo para la demo (K9 recalibration log)

def generar_reporte_k9(outlier_info, ajuste_k9):
    fecha = outlier_info["fecha_lc"]
    semana = outlier_info["semana_lc"]
    riesgos = ", ".join(outlier_info["riesgos_lc"])

    rep = f"""
K9 RE-CALIBRATION REPORT – Lunes Crítico ({fecha}, semana {semana})

1. Resumen del evento:
   - Evento compuesto de riesgo(s): {riesgos}
   - Near Miss severo activando control crítico.
   - Evento fuera de la predicción del modelo proactivo.

2. Señales previas detectadas por K9:
   - Zonas previas: {outlier_info['zona_prev']}
   - Tendencias previas: {outlier_info['tendencia_prev']}
   - Índice previo: {outlier_info['indice_prev']}

3. Interpretación:
   K9 identifica señales emergentes no capturadas por el modelo proactivo,
   incluyendo tendencias ascendentes y señales amarillas persistentes.
   El lunes crítico confirma la subestimación del riesgo.

4. Ajuste del modelo K9 para ciclos futuros:
   - Nuevo threshold amarillo: {ajuste_k9['T_yellow_new']}
   - Nuevo threshold rojo: {ajuste_k9['T_red_new']}
   - Ajuste de pesos internos:
     {ajuste_k9['pesos_k9_new']}

5. Conclusión:
   De presentarse un patrón similar en el futuro, K9 lo clasificará 
   anticipadamente en zona crítica, incluso si el ranking proactivo 
   lo mantiene fuera del Top 3.
"""
    return rep


print(generar_reporte_k9(outlier_info, ajuste_k9))


In [ ]:
# Celda 8.1 – Exportación de datasets STDE v3 a CSV

# Trayectorias diarias y semanales
df_trayectorias.to_csv(os.path.join(OUTPUT_DIR, "stde_trayectorias_diarias.csv"), index=False)
df_trayectorias_semana.to_csv(os.path.join(OUTPUT_DIR, "stde_trayectorias_semana.csv"), index=False)

# FDO
df_fdo_diario.to_csv(os.path.join(OUTPUT_DIR, "stde_fdo_diario.csv"), index=False)

# Eventos, observaciones, auditorías
df_eventos.to_csv(os.path.join(OUTPUT_DIR, "stde_eventos.csv"), index=False)
df_observaciones.to_csv(os.path.join(OUTPUT_DIR, "stde_observaciones.csv"), index=False)
df_auditorias.to_csv(os.path.join(OUTPUT_DIR, "stde_auditorias.csv"), index=False)

# Modelo proactivo + thresholds
df_proactivo_thresholds.to_csv(
    os.path.join(OUTPUT_DIR, "stde_proactivo_semanal_v4_4_thresholds.csv"),
    index=False
)

# Señales K9 + índice
df_k9.to_csv(os.path.join(OUTPUT_DIR, "stde_senales_k9.csv"), index=False)

print("Archivos exportados en:", OUTPUT_DIR)


In [ ]:
# Celda 8.2 – Checks de consistencia STDE v3

def check_ids_existentes(df, col, df_catalogo, col_id, nombre):
    ids_df = set(df[col].dropna().unique())
    ids_cat = set(df_catalogo[col_id].dropna().unique())
    faltantes = ids_df - ids_cat
    if faltantes:
        print(f"[ALERTA] {nombre}: IDs no encontrados en catálogo:", faltantes)
    else:
        print(f"[OK] {nombre}: todos los IDs existen en el catálogo.")


print("=== Checks de riesgos / roles / tareas / áreas ===")
check_ids_existentes(df_eventos, "riesgo_id", df_riesgos, "id_riesgo", "Riesgos en eventos")
check_ids_existentes(df_observaciones, "riesgo_id", df_riesgos, "id_riesgo", "Riesgos en observaciones")
check_ids_existentes(df_eventos, "id_area", df_areas, "id_area", "Áreas en eventos")
check_ids_existentes(df_observaciones, "id_area", df_areas, "id_area", "Áreas en observaciones")

print("\n=== Lunes crítico ===")
df_lc = df_eventos[df_eventos["es_lunes_critico"] == True]
print("Eventos lunes crítico:", df_lc.shape[0])
print(df_lc[["id_evento", "riesgo_id", "tipo_narrativo"]])

print("\n=== Degradación relativa R02 vs R01/R03 (criticidad semanal media) ===")
print(df_trayectorias_semana[["semana", "criticidad_R01_media", "criticidad_R02_media", "criticidad_R03_media"]])

print("\n=== Observaciones por área (rangos generales) ===")
print(df_observaciones.groupby("id_area")["id_observacion"].count())

print("\n=== AUF reactivas vs OCC- / NMS ===")
print("Total AUF reactivas:", df_auf_reactivas.shape[0])
print("Total OCC-:", df_observaciones[(df_observaciones['tipo_observacion']=='OCC') & (df_observaciones['estado']=='OCC-')].shape[0])
print("Total NMS:", df_eventos[df_eventos['tipo_narrativo']=='NMS'].shape[0])
